In [1]:
import os

if "src" not in os.listdir():
    os.chdir("../")

In [2]:
import pandas as pd
from tqdm import tqdm
from qdrant_client import QdrantClient, models
from transformers import AutoTokenizer
from langchain_community.llms import VLLMOpenAI

tqdm.pandas()
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_colwidth", 100)

# pd.set_option('display.height', 500)

/home/harddisk/projects/legal_ml/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = QdrantClient(url="http://localhost:6333")

COLLECTION_NAME = "questions"

In [4]:
res = client.scroll(collection_name=COLLECTION_NAME, limit=100)

In [5]:
for i in res[0]:
    print(i.payload["question"])
    print(i.payload["question"])

Организация покупает имущество предприятия, признанного несостоятельным (банкротом), в ходе конкурсного производства на открытых торгах. Необходимо ли предварительно направлять ходатайство в Антимонопольный комитет на получение в собственность...? (ст. 18 п.1 Закона О конкуренции). Допускаются ли к участию в торгах лица без Заключения Антимонопольного комитета об удовлетворении ходатайств? Повлечет ли ничтожность сделки отсутствие разрешения Антимонопольного комитета?
Организация покупает имущество предприятия, признанного несостоятельным (банкротом), в ходе конкурсного производства на открытых торгах. Необходимо ли предварительно направлять ходатайство в Антимонопольный комитет на получение в собственность...? (ст. 18 п.1 Закона О конкуренции). Допускаются ли к участию в торгах лица без Заключения Антимонопольного комитета об удовлетворении ходатайств? Повлечет ли ничтожность сделки отсутствие разрешения Антимонопольного комитета?
Летом прошлого года наша фирма приобрела в собственнос

In [6]:
sample_data_id = [
    19577190,
    19456459,
    19669390,
    19553598,
    19809030,
    18912493,
    18700061,
    18912827,
    19844768,
    19338438,
    19606364,
    19240564,
    18959139,
    19858181,
    19537157,
    19098632,
    19804825,
    19666639,
    19274835,
    19622104,
    18995654,
    19312803,
    19705482,
    19662531,
    19805461,
    19688508,
    19095243,
    19566535,
    18740241,
    19330152,
    18868660,
    19371104,
    18768905,
    19387444,
    18855886,
    18786269,
    19411020,
    19851438,
    19848671,
    18654192,
    19714651,
    19830842,
    19089783,
    18909091,
    19632932,
    19522837,
    19003971,
    18849682,
    19398937,
    19412992,
    19707799,
    19784700,
    18633121,
    19595322,
    19469282,
    19899840,
    18818657,
    18884329,
    19836854,
    19644220,
    19215042,
    19267638,
    18652544,
    19454517,
    19265402,
    18813876,
    18667803,
    19287233,
    19505180,
    18847431,
    19575379,
    18733335,
    19690287,
    18909389,
    19151901,
    19263633,
    18898442,
    19521973,
    19061900,
    19343391,
    18788259,
    19213812,
    18899714,
    19720215,
    19295131,
    19889931,
    19642768,
    18790006,
    19565721,
    19521155,
    19061317,
    19669340,
    19311051,
    19878642,
    19283027,
    19617324,
    18811603,
    18840165,
    18993859,
    19615169,
]
len(sample_data_id)

100

In [7]:
def create_prompt_dataset(tokenizer, system_prompt, question, answer, context):
    prompt = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": question,
        },
        {"role": "assistant", "content": answer},
    ]
    record = {}
    record["prompt"] = tokenizer.apply_chat_template(
                    prompt, tokenize=False, add_generation_prompt=False
                )
    record["prompt_not_answer"] = tokenizer.apply_chat_template(
                    prompt[:-1], tokenize=False, add_generation_prompt=True
                )
    record["answer"] = answer
    record["question"] = question
    
    if context:
        prompt = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": question,
        },
        {"role": "system", "content": context},
        {"role": "assistant", "content": answer},
    ]
        record["prompt_context"] = tokenizer.apply_chat_template(
                    prompt, tokenize=False, add_generation_prompt=False
                )
        record["prompt_context_not_answer"] = tokenizer.apply_chat_template(
                    prompt[:-1], tokenize=False, add_generation_prompt=True
                )
        record["context"] = context
    else:
        record["prompt_context"] = None
        record["prompt_context_not_answer"] = None
        record["context"] = None

    return record

model_name = "unsloth/Qwen2.5-7B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
system_prompt = (
    "Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, "
    "ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!"
)
question = "Хто я?"
answers = "Ты ***"
contexts = "Пример 1"

create_prompt_dataset(tokenizer, system_prompt, question, answers, contexts)

Could not cache non-existence of file. Will ignore error and continue. Error: [Errno 13] Permission denied: '/home/gleb/.cache/huggingface/hub/models--unsloth--Qwen2.5-7B-Instruct/.no_exist/f8f9da84f4cb89d2e749d6eef5b87e68255a8e9e/chat_template.jinja'


{'prompt': '<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>\n<|im_start|>user\nХто я?<|im_end|>\n<|im_start|>assistant\nТы ***<|im_end|>\n',
 'prompt_not_answer': '<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>\n<|im_start|>user\nХто я?<|im_end|>\n<|im_start|>assistant\n',
 'answer': 'Ты ***',
 'question': 'Хто я?',
 'prompt_context': '<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>\n<|im_start|>user\nХто я?<|im_end|>\n<|im_start|>system\nПример 1<|im_end|>\n<|im_start|>assistant\nТы ***<|im_end|>\n',
 'prom

In [6]:
system_prompt = (
    "Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, "
    "ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!"
)

payloads = []
for i in tqdm(sample_data_id):
    point = client.query_points(collection_name=COLLECTION_NAME, query=i, limit=1, with_vectors=True).points[0]
    payload, vector = point.payload, point.vector

    vect_search = client.query_points(
        collection_name=COLLECTION_NAME, query=vector, limit=4
    ).points[1:4]
    
    prav_sit = f"""Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\n"""
    for i in vect_search:
        prav_sit += f"""Вопрос гражданина:
    {i.payload["question"]}
    Ответ юриста:
    {i.payload["answer"]}
    """
    payloads.append({"system_prompt":system_prompt, "question":payload["question"], "answer":payload["answer"], "context":prav_sit})

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:03<00:00, 32.97it/s]


In [7]:
benchmark = []

for record in payloads:
    benchmark.append(create_prompt_dataset(tokenizer, **record))

In [8]:
benchmark = pd.DataFrame(benchmark)
benchmark.head(3)

,prompt,prompt_not_answer,answer,question,prompt_context,prompt_context_not_answer,context
0,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","Если военкомат отказал официально, обжалуйте его в суде.",Что делать если не берут а Армию? Я очень хочу служить России.,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...",Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\nВопр...
1,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","Да, можете просто взять и уйти из больницы. За такой уход вам ничего не будет.","Мне отказывают в написании отказа от госпитализации, могу ли я просто уйти из больницы?","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...",Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\nВопр...
2,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","Отпуск продлят ровно на то количество дней, на которые оформлен больничный. Ст. 124 ТК РФ. Если ...",У меня ежегодно оплачиваемый отпуск пришёлся на 25.07.22-21.08.22 (по 21 включительно) . Но 19.0...,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...",Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\nВопр...


In [9]:
path_save = "./data/interim/evalutions/benchmarks/100_prompt.csv"

pd.DataFrame(benchmark).to_csv(path_save, index=False)

In [10]:
path_save = "./data/interim/evalutions/benchmarks/100_prompt.csv"

benchmark = pd.read_csv(path_save)

In [11]:
benchmark.head(3)

,prompt,prompt_not_answer,answer,question,prompt_context,prompt_context_not_answer,context
0,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","Если военкомат отказал официально, обжалуйте его в суде.",Что делать если не берут а Армию? Я очень хочу служить России.,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...",Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\nВопр...
1,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","Да, можете просто взять и уйти из больницы. За такой уход вам ничего не будет.","Мне отказывают в написании отказа от госпитализации, могу ли я просто уйти из больницы?","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...",Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\nВопр...
2,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","Отпуск продлят ровно на то количество дней, на которые оформлен больничный. Ст. 124 ТК РФ. Если ...",У меня ежегодно оплачиваемый отпуск пришёлся на 25.07.22-21.08.22 (по 21 включительно) . Но 19.0...,"<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...","<|im_start|>system\nТы юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай,...",Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:\nВопр...


In [12]:
llm = VLLMOpenAI(
    openai_api_key="key",
    openai_api_base="http://localhost:5000/v1",
    model=model_name,
    max_tokens=1024,
)

In [13]:
result_prompt = llm.batch(benchmark["prompt_not_answer"].to_list())
result_prompt_context2 = llm.batch(benchmark["prompt_context_not_answer"].to_list())

In [18]:
index = 1
print("ВОПРОС:")
print(benchmark["prompt_context_not_answer"].to_list()[index])
print("ОТВЕТ:")
print(benchmark["answer"].to_list()[index])
print("ОТВЕТ МОДЕЛИ:")
print(result_prompt_context2[index])

ВОПРОС:
<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
Мне отказывают в написании отказа от госпитализации, могу ли я просто уйти из больницы?<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    Я лежу в наркологическом диспансере, могу я написать отказ от госпитализации я уйти домой?
    Ответ юриста:
    Ориентируйтесь на то, что медицинское вмешательство должно быть доброволнымФедеральный закон от 21.11.2011 N 323-ФЗ (ред. от 29.07.2017) "Об основах охраны здоровья граждан в Российской Федерации"""Статья 20. Информированное добровольное согласие на медицинское вмешательство и на отказ от медицинского вмешательства""1. Необходимым предварительным условием медицинского вмешательства является дача информированного добров

In [26]:
for index in range(benchmark.shape[0]):
    print("ВОПРОС:")
    print(benchmark["prompt_context_not_answer"].to_list()[index])
    print("ОТВЕТ:")
    print(benchmark["answer"].to_list()[index])
    print("ОТВЕТ МОДЕЛИ C КОНТЕКСТОМ:")
    print(result_prompt_context2[index])
    print("ОТВЕТ МОДЕЛИ БЕЗ КОНТЕКСТА:")
    print(result_prompt[index])
    print("_"*1000)

ВОПРОС:
<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
Что делать если не берут а Армию? Я очень хочу служить России.<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    Что делать если меня не берут в армию.
    Ответ юриста:
    Наслаждаться жизнью.А если вам так туда хочется-обжалуйте решение призывной комиссии.
    Вопрос гражданина:
    Я хочу в Армию но меня не берут. По причине не известно. Что делать. Очень хочу служить.
    Ответ юриста:
    Вам не сказали результаты комиссии?
    Вопрос гражданина:
    Я очень хочу служить в Россию.
    Ответ юриста:
    А вы гражданин какого государства?С уважением.
    <|im_end|>
<|im_start|>assistant

ОТВЕТ:
Если военкомат отказал официально, обжалуйте его в суде.
ОТВЕТ МОДЕ

In [24]:
for index in range(benchmark.shape[0]):
    print("ВОПРОС:")
    print(benchmark["prompt_context_not_answer"].to_list()[index])
    print("ОТВЕТ:")
    print(benchmark["answer"].to_list()[index])
    print("ОТВЕТ МОДЕЛИ:")
    print(result_prompt_context2[index])
    print("_"*1000)

ВОПРОС:
<|im_start|>system
Ты юрист. Ты консультируешь людей по разным проблемам, внимательно прочитай, что тебе пишут, ответь на поставленные вопросы и дай консультацию. Ответ пиши только на РУССКОМ языке!<|im_end|>
<|im_start|>user
Что делать если не берут а Армию? Я очень хочу служить России.<|im_end|>
<|im_start|>system
Перед тем как дать консультацию изучи вопросы других людей и ответы на них других юристов:
Вопрос гражданина:
    Что делать если меня не берут в армию.
    Ответ юриста:
    Наслаждаться жизнью.А если вам так туда хочется-обжалуйте решение призывной комиссии.
    Вопрос гражданина:
    Я хочу в Армию но меня не берут. По причине не известно. Что делать. Очень хочу служить.
    Ответ юриста:
    Вам не сказали результаты комиссии?
    Вопрос гражданина:
    Я очень хочу служить в Россию.
    Ответ юриста:
    А вы гражданин какого государства?С уважением.
    <|im_end|>
<|im_start|>assistant

ОТВЕТ:
Если военкомат отказал официально, обжалуйте его в суде.
ОТВЕТ МОДЕ

In [28]:
benchmark["model_answer_with_context"] = result_prompt_context2
benchmark["model_answer"] = result_prompt

In [35]:
model_name_save = model_name.replace("/", "_")
benchmark.to_csv(f"./data/interim/evalutions/result/100_prompt_{model_name_save}")